In [1]:
import json
import numpy as np
import os

In [2]:
from pinn import *
from nn import NN

torch.set_default_dtype(torch.float32)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
def serialize_savelist(keys: list, values : list, filename):
    dict = {k: v for k, v in zip(keys, values)}
    filename = f"{filename}.json"
    with open(filename, 'w') as file:
        json.dump(dict, file)

In [4]:
# Parameters model
rho = 0.5
mu = 79300 # MPa
lam = 100000 # MPa

Lx = 0.2
Ly = 0.2
T = 0.2

x_domain = np.array([0.0, Lx])/Lx
y_domain = np.array([0.0, Ly])/Lx
t_domain = np.array([0.0, T])/T

# Parameters NN
epochs = 12000
n_train = 30
layers = 2
dim_hidden = 40
lr = 0.02
om_INITIAL = 2
om_BOUNDARY = 0.4

# Eventually save changed $NN_2$ parameters to pass to other script

In [5]:
par = [Lx, T, n_train, layers, dim_hidden]
keys = ['x', 't', 'n', 'n_layers', 'n_neurons']

serialize_savelist(keys, par, 'par')

In [6]:
pinn = PINN(layers, dim_hidden, act=nn.Tanh()).to(device)

loss_fn = Loss(
    x_domain,
    y_domain,
    t_domain,
    n_train,
    return_adim(x_domain, t_domain, rho, mu, lam),
    initial_conditions,
    om_INITIAL,
    om_BOUNDARY
)

# Import weights from pretrained model

In [7]:
filename_model = get_last_modified_file('in_model')
pretrained_model_dict = torch.load(filename_model, map_location=torch.device(device))

pretrained_model = NN(layers, dim_hidden, 2, 1)
pretrained_model.load_state_dict(pretrained_model_dict)

<All keys matched successfully>

In [8]:
for i in np.arange(len(pinn.middle_layers)):
    pinn_layer = pinn.middle_layers[i]
    pretrained_layer = pretrained_model.middle_layers[i]
    pinn.middle_layers[i].weight.data.copy_(pretrained_model.middle_layers[i].weight)
    pinn.middle_layers[i].bias.data.copy_(pretrained_model.middle_layers[i].bias)

# Train new network with pretrained weights and biases in middle layers

In [9]:
pinn_trained, loss_values = train_model(
    pinn, loss_fn=loss_fn, learning_rate=lr, max_epochs=epochs)

Folder '2024-04-27' already exists.


Training:   0%|          | 0/12000 [00:00<?, ?it/s]

Parameter name: layer_in.weight
Parameter shape: torch.Size([40, 3])
Gradient:
tensor([[-5.8288e+11, -2.7312e+11, -2.0599e+11],
        [ 1.1265e+12,  6.6994e+11,  3.9808e+10],
        [ 1.4706e+12,  9.9825e+11,  3.7945e+11],
        [ 8.5162e+10,  3.4590e+10,  3.4695e+10],
        [-1.1241e+11, -5.2114e+10,  2.5819e+10],
        [-2.0214e+12, -1.0660e+12, -1.3579e+11],
        [-1.7294e+12, -8.1692e+11,  3.5926e+11],
        [-2.0623e+11, -9.2950e+10, -1.2432e+10],
        [ 1.3948e+12,  9.2302e+11,  3.6255e+11],
        [ 5.9268e+10,  1.9351e+10,  3.5547e+10],
        [ 6.1875e+10,  4.7277e+10,  7.6472e+10],
        [-7.6080e+11, -4.8484e+11,  8.3843e+10],
        [-1.0332e+11, -3.9405e+10, -6.7823e+10],
        [ 4.3742e+11,  1.9018e+11,  8.1403e+10],
        [-2.1921e+12, -1.1286e+12, -1.9030e+11],
        [ 3.9597e+11,  1.6737e+11,  1.5841e+10],
        [-1.5256e+12, -7.1127e+11,  1.8242e+11],
        [-4.4855e+11, -2.1496e+11, -1.0819e+11],
        [ 6.0196e+10,  4.2566e+09, -6.7

Loss: 188845924492682593809234657280.0000:   0%|          | 1/12000 [00:02<7:09:25,  2.15s/it]

Parameter name: layer_in.weight
Parameter shape: torch.Size([40, 3])
Gradient:
tensor([[-5.1724e+11, -2.5025e+11, -1.7946e+11],
        [ 1.0499e+12,  6.7618e+11,  3.6168e+10],
        [ 1.1245e+12,  9.0446e+11,  2.2014e+11],
        [ 7.1424e+10,  2.8774e+10,  2.8973e+10],
        [-1.2265e+11, -5.8790e+10,  1.8488e+10],
        [-1.8937e+12, -1.0492e+12, -1.0977e+11],
        [-1.6208e+12, -7.9500e+11,  4.0902e+11],
        [-1.9542e+11, -9.0586e+10, -1.0566e+10],
        [ 1.1598e+12,  8.5780e+11,  2.6073e+11],
        [ 8.0573e+10,  2.8369e+10,  4.8975e+10],
        [ 7.7640e+10,  5.8624e+10,  9.1556e+10],
        [-5.9890e+11, -4.5490e+11,  1.5741e+11],
        [-8.9587e+10, -3.0600e+10, -6.2548e+10],
        [ 4.3486e+11,  1.8921e+11,  8.9027e+10],
        [-2.0026e+12, -1.0728e+12, -1.2262e+11],
        [ 3.7983e+11,  1.6008e+11,  8.8472e+09],
        [-1.3923e+12, -6.6820e+11,  2.3392e+11],
        [-4.2511e+11, -2.1133e+11, -1.0775e+11],
        [ 8.2167e+10,  6.7719e+09,  1.0

...

Parameter name: layer_in.weight
Parameter shape: torch.Size([40, 3])
Gradient:
tensor([[-3.5636e+11, -1.6768e+11, -8.2202e+10],
        [ 9.8247e+11,  6.8774e+11,  8.3061e+10],
        [ 6.1964e+11,  8.6114e+11,  3.1256e+10],
        [ 3.8140e+10,  1.4087e+10,  1.2454e+10],
        [-1.9095e+11, -1.0171e+11, -4.0291e+10],
        [-1.8191e+12, -9.6037e+11, -7.9529e+10],
        [-2.1147e+12, -1.1231e+12, -9.1463e+09],
        [-1.9036e+11, -9.5081e+10, -2.4697e+10],
        [ 8.5615e+11,  8.0406e+11,  1.7415e+11],
        [ 8.6162e+10,  9.3659e+09,  4.0926e+10],
        [ 1.0881e+10, -1.5060e+09,  3.4056e+10],
        [-5.5010e+11, -5.8133e+11,  9.0829e+10],
        [-7.4255e+09,  4.9948e+10,  2.5855e+10],
        [ 4.2127e+11,  1.4825e+11,  8.4663e+10],
        [-1.8814e+12, -9.8526e+11, -6.9886e+10],
        [ 4.1277e+11,  1.5353e+11,  1.7927e+10],
        [-1.4661e+12, -6.3356e+11,  2.1377e+11],
        [-3.5103e+11, -1.5769e+11, -6.8930e+10],
        [ 1.3469e+11,  1.5591e+09,  3.3

Loss: 385653334343680.0000:   0%|          | 12/12000 [00:13<3:07:26,  1.07it/s]

Parameter name: layer_in.weight
Parameter shape: torch.Size([40, 3])
Gradient:
tensor([[-3.4986e+11, -1.6204e+11, -7.6370e+10],
        [ 9.7716e+11,  6.7844e+11,  8.0249e+10],
        [ 6.3002e+11,  8.6740e+11,  4.6392e+10],
        [ 3.7263e+10,  1.3570e+10,  1.2030e+10],
        [-1.9239e+11, -1.0243e+11, -4.1954e+10],
        [-1.8032e+12, -9.3393e+11, -5.7961e+10],
        [-2.1379e+12, -1.1277e+12, -2.7126e+10],
        [-1.9060e+11, -9.4927e+10, -2.5722e+10],
        [ 8.5935e+11,  8.0228e+11,  1.8088e+11],
        [ 8.2255e+10,  5.9215e+09,  3.6641e+10],
        [ 3.7337e+09, -7.3652e+09,  2.6801e+10],
        [-5.6607e+11, -5.9057e+11,  7.3811e+10],
        [-8.8075e+08,  5.6461e+10,  3.4419e+10],
        [ 4.1505e+11,  1.4149e+11,  7.8411e+10],
        [-1.8691e+12, -9.6338e+11, -5.2584e+10],
        [ 4.1074e+11,  1.4904e+11,  1.3855e+10],
        [-1.4604e+12, -6.1487e+11,  2.2865e+11],
        [-3.4458e+11, -1.5118e+11, -6.2610e+10],
        [ 1.3311e+11, -7.8796e+08,  3.1

Loss: 202335506661376.0000:   0%|          | 13/12000 [00:14<3:18:09,  1.01it/s]

KeyboardInterrupt: 

In [ ]:
current_time = get_current_time(fmt="%H:%M")

folder = 'model'
model_name = f'{lr}_{epochs}_{dim_hidden}_{current_time}.pth'
model_path = os.path.join(folder, model_name)
os.makedirs(folder, exist_ok=True)

torch.save(pinn_trained.state_dict(), model_path)

In [ ]:
from plots import plot_initial_conditions, plot_solution

x, y, _ = get_initial_points(x_domain, y_domain, t_domain, n_train)
t_value = 0.0
t = torch.full_like(x, t_value)
x = x.to(device)
y = y.to(device)
t = t.to(device)
z = f(pinn, x ,y, t)
ux_0, uy_0 = initial_conditions(x, y, Lx, i = 1)
z_0 = torch.cat((ux_0, uy_0), dim=1)

plot_initial_conditions(z_0, y, x, 'Initial conditions - analytical', n_train, from_pinn = 0)
plot_initial_conditions(z, y, x, 'Initial conditions - NN', n_train)

In [ ]:
from IPython.display import HTML

x, y, t = get_interior_points(x_domain, y_domain, t_domain, n_train)
animation = plot_solution(pinn, x, y, t, n_train)
HTML(animation.to_jshtml())

# To be added
- ~separate loss in more bars to see how the various loss term come to zero~
- see if some quadrature rule has been implemented
- scheme of weights initialization in order to automatically satisfy initial conditions
- plots (in progress)
- NN operators (to generalize results)
- try to implement function that allows that satisfy initial conditions?

# To be fully understood
- reshape when plotting (also different number of points allowed?)
- ~not consistent initial conditions~ (between $u_x$ and $u_y$)?